# Recover Current profiles from old Phase Diagram 

In [1]:
import sys
import os
sys.path.insert(0, '../magcolloids')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML, clear_output
import scipy.optimize as spo
import scipy.spatial as spa 
import magcolloids as mgc
import matplotlib as mpl
import string as st
import itertools as it
import support as sp

from tqdm.auto import tqdm

ureg = mgc.ureg

idx = pd.IndexSlice

%reload_ext autoreload
%autoreload 2

In [2]:
mpl.rc('text', usetex=True)
mpl.rcParams['figure.dpi'] = 150

### To add errorbars to this figure I need to recalculate the instantaneous current. 

In [3]:
directory = "/media/Antonio/Dynamic/DimerCurrents/FrequencySweeps/FrequencySweep_3.9/"
index = pd.read_csv(os.path.join(directory, "index.dat"), 
                    sep=" ", names = ["file", "h", "tilt"])

In [4]:
index.head()

,file,h,tilt
0,Tilt_2.0_deg_h3899_2019_09_25_17_17_30,3.9,2.0
1,Tilt_16.5_deg_h3899_2019_09_25_17_17_30,3.9,16.5
2,Tilt_4.5_deg_h3899_2019_09_25_17_17_30,3.9,4.5
3,Tilt_1.5_deg_h3899_2019_09_25_17_17_30,3.9,1.5
4,Tilt_14.0_deg_h3899_2019_09_25_17_17_30,3.9,14.0


In [5]:
fmax = 7.5*ureg.Hz # 7.5Hz
dt = 15*ureg.sec # 15sec
df = 0.125*ureg.Hz

ratio = df/dt
total_time = fmax/ratio

f = st.Template("$ratio*ceil(time/$dt)*$dt").substitute(
    dt = dt.to(ureg.us).magnitude,
    ratio = ratio.to((ureg.MHz/ureg.us)).magnitude) * ureg.MHz

frequency = lambda time: (ratio.to((ureg.MHz/ureg.us))*\
                          (np.ceil(time.to(ureg.us)/dt.to(ureg.us))*dt)).to(ureg.Hz)
tilt_angle = np.concatenate([np.arange(0,30.5,0.5)])

In [6]:
def subset(times, dt, rnge):
    """ Defines a repeated range of times. 
    It returns the same range for every value of frequency"""
    fun = lambda times: (np.mod(times[:],dt))/dt
    thresh1 = (min(rnge))/dt
    thresh2 = (max(rnge))/dt
    return (fun(times)>thresh1) & (fun(times)<thresh2)

    def differentiated_velocity(name):
        ## Lazy open the trajectory
        trj_read = mgc.trj_lazyread(os.path.join(directory,name.file+".lammpstrj"), 
                         output = ["x","y","z","mux","muy","muz","fx","fy"])

        ## Define a subset
        timestep = 1e-4*ureg.s
        frames = np.array(list(trj_read.T.keys()))
        times = frames*timestep
        readthis = subset(times, dt, [7*ureg.s,14*ureg.s])

        ## Load Trajectory
        trj = trj_read.read_trj(np.where(readthis))
        bounds = trj_read.get_bounds(slice(0,1,1))
        trj = trj.filter(["x","y","z"])
        trj["time"] = trj.index.get_level_values("frame")*timestep.to(ureg.s).magnitude
        trj["frequency"] = frequency(trj.time.values*ureg.s).to(ureg.Hz).magnitude

        ## Calculate Velocity Array
        velocities = sp.calculate_velocities(trj, bounds)
        velocities["frequency"] = trj["frequency"]

        ## Calculate Differentiated Velocity
        velocities["plane"] = np.NaN
        velocities.loc[trj.z>0,"plane"] = "up"
        velocities.loc[trj.z<0,"plane"] = "down"

        vupdown = velocities.groupby(["plane","frequency"]).mean().filter(["x","y"])

        vupdown = vupdown.join(
            velocities.groupby(["plane","frequency"]
                          ).sem().filter(["x","y"]).rename(
                columns = dict(x="x_err", y = "y_err")))

        vupdown = vupdown.join(
            velocities.groupby(["plane","frequency"]
                          ).std().filter(["x","y"]).rename(
                columns = dict(x="x_std", y = "y_std")))

        vupdown.to_csv(os.path.join(directory,name.file+"_vupdown_w_err.dat"), sep="\t")


        return vupdown

In [7]:
def differentiated_velocity(name):
    ## Lazy open the trajectory
    trj_read = mgc.trj_lazyread(os.path.join(directory,name.file+".lammpstrj"), 
                     output = ["x","y","z","mux","muy","muz","fx","fy"])

    ## Define a subset
    timestep = 1e-4*ureg.s
    frames = np.array(list(trj_read.T.keys()))
    times = frames*timestep
    readthis = subset(times, dt, [7*ureg.s,14*ureg.s])

    ## Load Trajectory
    trj = trj_read.read_trj(np.where(readthis))
    bounds = trj_read.get_bounds(slice(0,1,1))
    trj = trj.filter(["x","y","z"])
    trj["time"] = trj.index.get_level_values("frame")*timestep.to(ureg.s).magnitude
    trj["frequency"] = frequency(trj.time.values*ureg.s).to(ureg.Hz).magnitude

    ## Calculate Velocity Array
    velocities = sp.calculate_velocities(trj, bounds)
    velocities["frequency"] = trj["frequency"]

    ## Calculate Differentiated Velocity
    velocities["plane"] = np.NaN

    velocities["x"] = velocities["x"] * np.sign(trj.z)
    velocities["y"] = velocities["y"] * np.sign(trj.z)

    vel_inst = velocities.groupby("frame").mean()

    vupdown = vel_inst.groupby(["frequency"]).mean().filter(["x","y"])

    vupdown = vupdown.join(
        vel_inst.groupby(["frequency"]
                      ).sem().filter(["x","y"]).rename(
            columns = dict(x="x_err", y = "y_err")))

    vupdown = vupdown.join(
        vel_inst.groupby(["frequency"]
                      ).std().filter(["x","y"]).rename(
            columns = dict(x="x_std", y = "y_std")))

    vupdown.to_csv(os.path.join(directory,name.file+"_vupdown_w_err.dat"), sep="\t")
    
    return vupdown

In [8]:
from multiprocessing import Pool

In [9]:
%%time
if __name__ ==  '__main__': 
    num_processors = 6
    p=Pool(processes = num_processors)
    diff_vel = list(tqdm(
            p.imap(differentiated_velocity, 
                   [name for i,name in index.iterrows()]), 
            total=len(index)))

  0%|          | 0/61 [00:00<?, ?it/s]

CPU times: user 340 ms, sys: 120 ms, total: 461 ms
Wall time: 1h 59min 59s


In [11]:
diff_vel_pd = pd.concat(diff_vel, keys = index.index, names=["file"])
diff_vel_pd.to_csv(os.path.join(directory,"differentiated_velocities_w_err.dat"),
                   sep = "\t")